# Что нужно сделать 
1. Поработать с ячейкой formula_combined, удалить structure_
2. Геометрические признаки (объём, площадь, сферичность)
3. Total_magnetization по формуле (сумма 2-3)/const (из примера)
3. Pymatgen [
1)  Массовое соотношение атомов
2)  Number of magnetic ions per unit cell
3) get_exchange_group_info
4) number_of_unique_magnetic_sites
5) number_of_magnetic_sites]

In [76]:
# All library connection
import pandas as pd
import numpy as np
import re #регулярки
from math import gcd, isclose
import sympy as sp #библиотека символьных вычеслений, мат.задачи
import matplotlib.pyplot as plt
import seaborn as sns

In [77]:
from pymatgen.core.structure import Composition

In [78]:
df = pd.read_csv(r'C:\Users\Ксения\Project\Magnet_Exchange_bias\data\Bias_data_1.csv')

In [79]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         533 non-null    int64  
 1   c1                 533 non-null    object 
 2   c2                 322 non-null    object 
 3   c3                 9 non-null      object 
 4   chemical_formula   533 non-null    object 
 5   crystal_structure  533 non-null    object 
 6   depth_nm           509 non-null    float64
 7   id                 533 non-null    int64  
 8   space_group_shell  533 non-null    int64  
 9   structure          533 non-null    object 
 10  structure_         533 non-null    object 
 11  formula_combined   533 non-null    object 
 12  coer_oe            533 non-null    float64
 13  exc_bias_oe        533 non-null    float64
 14  fc_field_t         533 non-null    float64
 15  h_range_max_koe    533 non-null    float64
 16  length_d_nm        533 non

# Исправим стотлбец formula_combined

Проблема была в том, что формулы, что должны были иметь вид Li2 Cr2 O4 имели вид Li2 Cr1 O3 Cr1 O1

In [80]:
print(df['formula_combined'].unique()) # Пример, что нас не устраивает

['Zn1 O1 Fe2 Co1 O4' 'Co1 O1 Fe2 Co1 O4' 'Mn1 Zn1 Fe4 O8' 'La1 Fe1 O3'
 'Ni1 Ni1 O1' 'Mg1 Fe2 O4' 'Fe2 Co1 O4' 'Fe1 Bi1 O3'
 'Fe2 Co1 O4 Fe1 Bi1 O3' 'Co1 Co1 O1' 'Fe1 Rh1' 'Mn1 O2' 'Ag1 Fe3 O4'
 'Fe1 Fe2 O3' 'Fe1 O1 Fe3 O4' 'Ni1 O1' 'Fe3 O4' 'Fe3 O4 Ni1 O1'
 'Ni1 O1 Fe3 O4' 'Fe4 Bi2 O9 Fe1 Bi1 O3' 'Fe1 Bi1 O3 Fe4 Bi2 O9'
 'Ti1 O2 Fe2 O3' 'Fe13 Co2 O20' 'Fe23 Co7 O40' 'Fe7 Co3' 'Fe1 Co1'
 'Fe3 Co7' 'Fe1 Fe3 O4' 'La2 Mn1 Fe1 O6' 'V1 Fe1 O4 Fe3 O4' 'Au1 Fe3 O4'
 'Gd1 Ti1 Fe9 Bi9 O30' 'Sr11 Nd39 Co50 Cu97 O150' 'Fe3 O4 Co1 O1'
 'Fe2 Co1 O4 Co1 O1' 'Cu98 Fe1 O50' 'Cu96 Fe1 O25' 'Fe2 O3'
 'Sr3 La17 Fe20 O60' 'Co1 Co3 O4' 'Mn3 O4' 'Sr33 La67 Mn100 O300'
 'Mn1 O1 Mn3 O4' 'Fe3 O4 Fe2 O3' 'Co3 O4' 'Fe1 Ni99 O100' 'Fe1 Ni49 O50'
 'Fe1 Ni24 O25' 'Fe3 Ni97 O100' 'Cr5 Ag1 O8' 'Cr1 Ag1 O2'
 'Co1 O1 Fe2 Ni1 O4' 'Mn1 Fe2 O4' 'Mn1 Fe2 O4 Fe3 O4' 'La1 Fe10 Bi9 O30'
 'Nd1 Fe1 O3' 'Fe2 Co1 O4 Fe3 O4' 'Dy1 Ti1 Cr1 O5' 'Gd1 Ti1 Cr1 O5'
 'Ni1 O1 Mn2 Co1 O4' 'Ba1 Fe12 O19 Fe3 O4' 'Fe2 Ni1 O4 Co1 O1'
 'Cr2 Co1 

Пример работы с такой ошибкой

1. Создание объекта Composition из вашей формулы
comp = Composition("Zn1 O1 Fe2 Co1 O4") # Может теперь можно склеить их обратно

2. Получение словаря количества атомов каждого элемента
el_amt_dict = comp.get_el_amt_dict()

3. Собираем обратно химическую формулу из словаря
formula = ""
for el, amt in el_amt_dict.items():
    formula += f"{el}{'' if amt == 1 else str(int(amt))}"

print(Composition(formula))

In [85]:
def transform_formula(formula):
    '''
    Zn1 O1 Fe2 Co1 O4 -> {'Zn': 1.0, 'O': 5.0, 'Fe': 2.0, 'Co': 1.0} -> Zn1 O5 Fe2 Co1
    '''
    comp = Composition(formula)
    el_amt_dict = comp.get_el_amt_dict() # Получение словаря с количеством атомов в формуле
    # Обратная сборка
    new_formula = ''
    for el, amt in el_amt_dict.items():
        new_formula += f"{el}{'' if amt == 1 else str(int(amt))}"
    return new_formula

In [86]:
def process_column(df, column_name):
    """
    Функция проверяет каждую ячейку в столбце DataFrame на равенство 0 или NaN.
    Если значение не равно 0 или NaN, оно заменяется на результат применения класса Composition из библиотеки pymatgen.

    Args:
        df (pandas.DataFrame): Исходный DataFrame.
        column_name (str): Имя столбца, который нужно обработать.

    Returns:
        pandas.DataFrame: DataFrame c обработанным столбцом.
    """
    new_column = []
    for value in df[column_name]:
        if pd.notna(value) and value != 0:
            # Если значение не NaN и не равно 0, применяем класс Composition
            comp = Composition(str(value))
            new_value = comp.formula
        else:
            new_value = value
        new_column.append(new_value)

    df[column_name] = new_column
    return df

In [87]:
df['formula_combined'] = df['formula_combined'].apply(transform_formula)
df = process_column(df, 'formula_combined')
print(df['formula_combined'].unique()) # Тепрь у нас есть полный средний материал ядра+оболочки

['Zn1 Fe2 Co1 O5' 'Fe2 Co2 O5' 'Mn1 Zn1 Fe4 O8' 'La1 Fe1 O3' 'Ni2 O1'
 'Mg1 Fe2 O4' 'Fe2 Co1 O4' 'Fe1 Bi1 O3' 'Fe3 Co1 Bi1 O7' 'Co2 O1'
 'Fe1 Rh1' 'Mn1 O2' 'Fe3 Ag1 O4' 'Fe3 O3' 'Fe4 O5' 'Ni1 O1' 'Fe3 O4'
 'Fe3 Ni1 O5' 'Fe5 Bi3 O12' 'Ti1 Fe2 O5' 'Fe13 Co2 O20' 'Fe23 Co7 O40'
 'Fe7 Co3' 'Fe1 Co1' 'Fe3 Co7' 'Fe4 O4' 'La2 Mn1 Fe1 O6' 'V1 Fe4 O8'
 'Fe3 Au1 O4' 'Gd1 Ti1 Fe9 Bi9 O30' 'Sr11 Nd39 Co50 Cu97 O150'
 'Fe3 Co1 O5' 'Fe1 Cu98 O50' 'Fe1 Cu96 O25' 'Fe2 O3' 'Sr3 La17 Fe20 O60'
 'Co4 O4' 'Mn3 O4' 'Sr33 La67 Mn100 O300' 'Mn4 O5' 'Fe5 O7' 'Co3 O4'
 'Fe1 Ni99 O100' 'Fe1 Ni49 O50' 'Fe1 Ni24 O25' 'Fe3 Ni97 O100'
 'Cr5 Ag1 O8' 'Cr1 Ag1 O2' 'Fe2 Co1 Ni1 O5' 'Mn1 Fe2 O4' 'Mn1 Fe5 O8'
 'La1 Fe10 Bi9 O30' 'Nd1 Fe1 O3' 'Fe5 Co1 O8' 'Dy1 Ti1 Cr1 O5'
 'Gd1 Ti1 Cr1 O5' 'Mn2 Co1 Ni1 O5' 'Ba1 Fe15 O23' 'Cr2 Co1 O4' 'Fe1 Ni1'
 'Mn4 O4' 'Mn1 Fe3 O5' 'Fe4 C1' 'Mg1 Fe3 O5' 'Mg1 Fe5 Co1 O9' 'Fe2 Co1 O3'
 'Fe4 Co1 Ni1 O8' 'Fe2 Ni1 O4' 'Mn1 Fe4 O7' 'Fe8 C2 O4' 'Fe5 C2' 'Fe1 Ni3'
 'Fe1 Ni3 Mo1' 'Fe11 C5' 'Fe11 

# Добавление дескрипторов

// TODO: добавление параемтров обязательно 
1. Массовое соотношение атомов +
2. Number of magnetic ions per unit cell
3. get_exchange_group_info
4. number_of_unique_magnetic_sites
5. number_of_magnetic_sites

In [91]:
def calculate_mass_ratio(formula):
    """
    Рассчитывает массовое соотношение атомов в химической формуле для наших соединений

    Args:
        formula (str): Химическая формула.

    Returns:
        float: Массовое соотношение атомов.
    """
    comp = Composition(formula)
    elements = comp.elements
    atomic_masses = [element.atomic_mass for element in elements]
    total_mass = sum(atomic_masses)
    mass_ratios = [mass / total_mass for mass in atomic_masses]
    return mass_ratios

In [92]:
# Применяем функцию к столбцу с испаравлеными полными формулами материалов
df['mass_ratio'] = df['formula_combined'].apply(calculate_mass_ratio)
print(df['mass_ratio'])

0      [0.3334019839632774 amu, 0.28465247587379755 a...
1      [0.42702268687537803 amu, 0.4506367852995003 a...
2      [0.2858506267019325 amu, 0.34033252624746124 a...
3      [0.2858506267019325 amu, 0.34033252624746124 a...
4      [0.2858506267019325 amu, 0.34033252624746124 a...
                             ...                        
528    [0.5155277027331209 amu, 0.20726069721466786 a...
529    [0.5155277027331209 amu, 0.20726069721466786 a...
530    [0.5155277027331209 amu, 0.20726069721466786 a...
531    [0.5155277027331209 amu, 0.20726069721466786 a...
532    [0.5155277027331209 amu, 0.20726069721466786 a...
Name: mass_ratio, Length: 533, dtype: object


## Геометрические признаки

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         533 non-null    int64  
 1   c1                 533 non-null    object 
 2   c2                 322 non-null    object 
 3   c3                 9 non-null      object 
 4   chemical_formula   533 non-null    object 
 5   crystal_structure  533 non-null    object 
 6   depth_nm           509 non-null    float64
 7   id                 533 non-null    int64  
 8   space_group_shell  533 non-null    int64  
 9   structure          533 non-null    object 
 10  structure_         533 non-null    object 
 11  formula_combined   533 non-null    object 
 12  coer_oe            533 non-null    float64
 13  exc_bias_oe        533 non-null    float64
 14  fc_field_t         533 non-null    float64
 15  h_range_max_koe    533 non-null    float64
 16  length_d_nm        533 non

In [ ]:
df.rename(columns={'width_nm': 'x', 'length_d_nm': 'y', 'depth_nm': 'z'}, inplace=True)  # Для удобства теперь x, y, z

Помним что все наши формы закодированы:
'''
{
    1: ['sphere', 'quasi-spherical', 'quasi-sphere', 'pseudo-spherical', 'pseudospherically', 'egg', 'flower'], 
    2: ['octahedral', 'octahedron', 'octahedral '],  
    3: ['cubic', 'cube'],
    4: ['hexahedron', 'hexagonal'],
    5: ['tetrahedral', 'triangle'],
    6: ['wire', 'rod'], 
    7: ['octopod']
}
'''

In [ ]:
print(df['shape'].unique())

[1. 5. 2. 6. 3. 7. 4.]


In [ ]:
# Добавим колонки для наших значений 
df.insert(0, 'area', 0)
df.insert(0, 'volume', 0)
df = df.astype({'volume': float ,'area': float})

In [ ]:
# Создадим словари для подсчёта площади и объёма
dict_area = {
    1: lambda x, y=1, z=1: 12.57 * x ** 2, 
    2: lambda x, y=1, z=1: 1.732 * 2 * x ** 2, 
    3: lambda x, y, z=1: 6 * x * y, 
    4: lambda x, y=1, z=1: 2 * 2.60 * x ** 2 + 6 * x * z/2,   # TODO: #Разделить шестиугольную призму и hexahedron
    5: lambda x, y=1, z=1: 1.73 * x ** 2, 
    6: lambda x, y, z=1: 2* 3.14 *((x/2) + y), 
    7: lambda x, y=1, z=1: 3.46 * x ** 2   
}

dict_volume = {
    1: lambda x, y=1, z=1: 4.19 * x ** 3, 
    2: lambda x, y=1, z=1: 1.4142 * 0.3333 * x ** 3, 
    3: lambda x, y, z: x * y * z, 
    4: lambda x, y=1, z=1: 2.60 * x ** 2 * z/2,  # TODO: разделить формы 
    5: lambda x , y=1, z=1: 0.118 * x ** 3,  
    6: lambda x, y, z=1: 3.14 * x/4 * y **2,
    7: lambda x, y=1, z=1: 0.47 * x ** 3
}

In [ ]:
# Создаем функцию для вычисления площади
def calculate_area(row):
    shape = float(row['shape'])
    x = row['x']
    y = row.get('y', 0)
    z = row.get('z', 0)
    
    if shape in dict_area:
        if shape in [1, 2, 3, 4, 5, 6, 7]:
            return dict_area[shape](x, y) if shape in [3, 6] else dict_area[shape](x, z) if shape == 4 else dict_area[shape](x)
    else:
        return 'error'

# Создаем функцию для вычисления объема
def calculate_volume(row):
    shape = float(row['shape'])
    x = row['x']
    y = row.get('y', 0)
    z = row.get('z', 0)
    
    if shape in dict_volume:
        if shape in [1, 2, 3, 4, 5, 6, 7]:
            return dict_volume[shape](x, y, z) if shape in [3, 6] else dict_volume[shape](x, z) if shape == 4 else dict_volume[shape](x)
    else:
        return 'error'

In [ ]:
df['area'] = df.apply(calculate_area, axis=1)
df['volume'] = df.apply(calculate_volume, axis=1)

In [ ]:
print(df['area'].unique())

[6.88333200e+02 6.15930000e+02 3.63273000e+03 1.52097000e+03
 1.81008000e+03 4.07268000e+03 5.54337000e+03 9.16353000e+03
 1.62907200e+04 1.13130000e+04 2.49120000e+02 4.42880000e+02
 1.99526400e+03 3.77229600e+03 3.93805530e+03 5.23113120e+03
 7.42245930e+03 1.36058937e+04 7.85625000e+03 5.02800000e+05
 6.34280000e+03 5.31082500e+04 1.70603183e+03 3.76207530e+03
 2.50477800e+02 1.36887300e+04 3.40154306e+04 1.26713456e+04
 4.53777000e+03 2.12433000e+03 2.09177370e+03 2.82825000e+03
 3.54775680e+03 4.34871720e+03 5.02800000e+03 1.21500000e+04
 1.38240000e+04 1.56060000e+04 2.40000000e+03 3.21792000e+03
 2.16250000e+03 6.78944000e+02 6.14400000e+03 7.35000000e+03
 2.11925000e+03 2.39383080e+03 1.25700000e+03 8.04480000e+02
 1.12233600e+03 1.60506330e+03 4.02755370e+03 6.82412730e+03
 1.34393312e+04 8.15570239e+03 2.01120000e+04 2.43355200e+04
 3.01805700e+04 5.14867200e+04 1.80175816e+04 2.32419300e+04
 1.78003770e+03 1.38584250e+03 1.20722280e+03 9.95669700e+04
 1.01817000e+05 5.710249

## Pymatgen